In [1]:
import numpy as np
import pandas as pd
from collections import Counter
import itertools

In [2]:
# download data
# tinyurl generated via http://www.genenames.org/cgi-bin/download?col=gd_hgnc_id&col=gd_app_sym&col=gd_app_name&col=gd_prev_sym&col=gd_aliases&col=gd_pub_eg_id&col=gd_pub_ensembl_id&col=gd_pub_refseq_ids&col=md_prot_id&status=Approved&status_opt=2&where=&order_by=gd_hgnc_id&format=tiny&limit=&hgnc_dbtag=on&submit=submit
tmp = pd.read_table(
    "http://www.genenames.org/cgi-bin/download?col=gd_hgnc_id;col=gd_app_sym;col=gd_app_name;col=gd_prev_sym;col=gd_aliases;col=gd_pub_eg_id;col=gd_pub_ensembl_id;col=gd_pub_refseq_ids;col=md_prot_id;status=Approved;status_opt=2;where=;order_by=gd_hgnc_id;format=text;limit=;hgnc_dbtag=on;submit=submit",
   dtype="unicode"
)

In [3]:
tmp.columns =[
    u'HGNC_ID', u'Symbol', u'Name', u'Previous_Symbols',
    u'Synonyms', u'Entrez', u'Ensembl', u'RefSeq',
    u'UniProt']

In [4]:
for i,x in enumerate(tmp.columns):
    print i,x


0 HGNC_ID
1 Symbol
2 Name
3 Previous_Symbols
4 Synonyms
5 Entrez
6 Ensembl
7 RefSeq
8 UniProt


In [5]:
def uniq_mapping(df,col1,col2):
    uniqC1 = [k for k,v in Counter(df[col1]).iteritems() if v==1]
    uniqC2 = [k for k,v in Counter(df[col2]).iteritems() if v==1]
    uniqC1_idx = df.reset_index().set_index(col1).loc[uniqC1]["index"].values
    uniqC2_idx = df.reset_index().set_index(col2).loc[uniqC2]["index"].values
    return df.loc[np.intersect1d(uniqC1_idx,uniqC2_idx)]

In [6]:
for col1,col2 in list(itertools.combinations(tmp.columns[[1,5,6,7,8]].values,2)):
    df = tmp[[col1,col2]].dropna()
    udf = uniq_mapping(df,col1,col2)
    assert udf.shape[0]== np.unique(udf[col1].values).shape[0]
    assert udf.shape[0]== np.unique(udf[col2].values).shape[0]
    udf.to_pickle("%s2%s.p"%(col1,col2))

### GeneBank to Entrez via BioMart
Table obtained via [online tool](http://www.ensembl.org/biomart).

In [15]:
tmp = pd.read_table("GenBank2Entrez.txt",dtype=unicode).iloc[:,[2,1]].dropna().drop_duplicates()

In [17]:
tmp.columns=["GeneBank","Entrez"]

In [19]:
tmp.to_pickle("GeneBank2Entrez.p")